<a href="https://colab.research.google.com/github/rishi15-t/PP-MM/blob/master/PP_MM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
dataset = pd.read_pickle('/content/drive/My Drive/dataset/w2v_vgg_embeddings.pkl')

In [0]:
import torch
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


def Train_Test_Val_Split(data , test_data_fraction = 0.3, val_data_fraction = 0.1) :
    
  
    data_genres_one_hot_encoding = pd.DataFrame.from_items(zip(data['genres'].index, data['genres'].values)).T
    Label_names = np.array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western'])
    data_genres_one_hot_encoding.columns = Label_names
    Data_train, Data_test, Labels_train, Labels_test = train_test_split(data, data_genres_one_hot_encoding, test_size = test_data_fraction)

    Data_train, Data_val, Labels_train, Labels_val = train_test_split(Data_train, Labels_train, test_size = val_data_fraction)

    Data_train = Data_train.reset_index(drop=True)
    Data_test = Data_test.reset_index(drop=True)
    Data_val = Data_val.reset_index(drop=True)
    
    Labels_train = torch.tensor(Labels_train.values)
    Labels_test = torch.tensor(Labels_test.values)
    Labels_val = torch.tensor(Labels_val.values)
    
    return (Data_train, Data_test, Data_val, Labels_train, Labels_test, Labels_val, Label_names)
    
Data_train, Data_test, Data_val, Labels_train_tensor, Labels_test_tensor, Labels_val_tensor, Label_names = Train_Test_Val_Split(dataset)

Data_train_tensor_text = torch.tensor(Data_train['w2v_embeddings'])
Data_test_tensor_text = torch.tensor(Data_test['w2v_embeddings'])
Data_val_tensor_text = torch.tensor(Data_val['w2v_embeddings'])

Data_train_tensor_image = torch.tensor(Data_train['vgg16_embeddings'])
Data_test_tensor_image = torch.tensor(Data_test['vgg16_embeddings'])
Data_val_tensor_image = torch.tensor(Data_val['vgg16_embeddings'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  # Remove the CWD from sys.path while we load stuff.


In [0]:
!pip install git+https://github.com/uber/pyro.git

  Cloning https://github.com/uber/pyro.git to /tmp/pip-req-build-vmlvmstc
  Running command git clone -q https://github.com/uber/pyro.git /tmp/pip-req-build-vmlvmstc
     |████████████████████████████████| 61kB 5.2MB/s 
  Created wheel for pyro-ppl: filename=pyro_ppl-1.2.1+74110869-cp36-none-any.whl size=476778 sha256=5d4c203ed201b4520dd5b7e388ae3e7d6798af1a2f7f2825aecaecf5a49859ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-1dogowyd/wheels/3c/10/ed/73d2332a097e2e9dc7d601ca2a99b0b4f9634e393474b78137
Successfully built pyro-ppl
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import torch
from torch.utils.data import IterableDataset, Dataset, DataLoader
from torch import autograd, nn, tanh, optim
import torch.nn.functional as F
from torchvision import transforms
import h5py
import numpy as np 
from pathlib import Path
import pyro
from pyro.distributions import Normal, Categorical, Laplace
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam


# model

class GMU_PP(nn.Module):

    def __init__(self, num_maxout_units = 2, hidden_layer_size = 512, text_embeddings_size = 300, img_embeddings_size = 4096, num_labels = 23, hidden_activation = None, dropout = 0.1):

        super(GMU_PP, self).__init__()
        self.num_labels = num_labels
        self.hidden_layer_size = hidden_layer_size
        self.text_embeddings_size = text_embeddings_size
        self.img_embeddings_size = img_embeddings_size

        self.linear_h_text = torch.nn.Linear(text_embeddings_size, self.hidden_layer_size, bias = False)
        self.linear_h_image = torch.nn.Linear(img_embeddings_size, self.hidden_layer_size, bias = False)
        self.linear_z = torch.nn.Linear(text_embeddings_size + img_embeddings_size, self.hidden_layer_size, bias = False)

        self.tanh = torch.nn.Tanh()
        self.sigmoid = torch.nn.Sigmoid()

        self.dropout = nn.Dropout(p=dropout)
        self.linear = torch.nn.Linear(self.hidden_layer_size, self.num_labels)
        
        #self.maxout = Maxout_MLP(self.hidden_layer_size, self.hidden_layer_size, dropout, num_maxout_units=num_maxout_units)

        #Added one hidden layer in between GMU units and output layer for testing
        self.linear_test = torch.nn.Linear(self.hidden_layer_size, self.hidden_layer_size)
        self.relu_test = torch.nn.ReLU()

    def forward(self, image_embeddings, text_embeddings):
        
        image_h = self.linear_h_image(image_embeddings)
        image_h = self.tanh(image_h)
        text_h = self.linear_h_text(text_embeddings)
        text_h = self.tanh(text_h)
        concat = torch.cat((image_embeddings, text_embeddings), 1)
        z = self.linear_z(concat)
        z = self.sigmoid(z)
        gmu_output = z*image_h + (1-z)*text_h
        
        #maxout_mlp_output = self.maxout(gmu_output)

        test = self.linear_test(gmu_output)
        test = self.relu_test(test)
        logits = self.linear(test)
        
        #logits = self.linear(maxout_mlp_output)
        
        if(self.training) :
            return logits
        else :
            output = self.sigmoid(logits)
            return output

    '''
    # pyro model
    def model(self, image_embeddings, text_embeddings, labels):

        
        def model_dist(*shape):
            loc = torch.zeros(*shape).cuda()
            scale = torch.ones(*shape).cuda()
            return Laplace(loc, scale)
        
        priors = {
            'linear_h_text.weight': model_dist(self.hidden_layer_size, self.text_embeddings_size), #no bias
            'linear_h_image.weight': model_dist(self.hidden_layer_size, self.img_embeddings_size), #no bias
            'linear_z.weight': model_dist(self.hidden_layer_size, self.text_embeddings_size + self.img_embeddings_size), #no bias
            'linear_test.weight': model_dist(self.hidden_layer_size, self.hidden_layer_size), 'linear_test.bias': model_dist(self.hidden_layer_size),
            'linear.weight': model_dist(self.num_labels, self.hidden_layer_size), 'linear.bias': model_dist(self.num_labels)
            }
        
        lifted_module = pyro.random_module("net", net, priors)
        lifted_reg_model = lifted_module().cuda()
        lhat = lifted_reg_model(image_embeddings, text_embeddings)
        pyro.sample("obs", Categorical(logits=lhat), obs=labels)

    # pyro guide
    def guide(self, image_embeddings, text_embeddings, labels):

        def infer_dist(name, *shape):
            l = torch.empty(*shape, requires_grad=True).cuda()
            s = torch.empty(*shape, requires_grad=True).cuda()
            torch.nn.init.normal_(l, std=0.01)
            torch.nn.init.normal_(s, std=0.01)
            loc = pyro.param(name+"_loc", l)
            scale = nn.functional.softplus(pyro.param(name+"_scale", s))
            return Laplace(loc, scale)

        dists = {
            'linear_h_text.weight': infer_dist("W1", self.hidden_layer_size, self.text_embeddings_size), #no bias
            'linear_h_image.weight': infer_dist("W2", self.hidden_layer_size, self.img_embeddings_size), #no bias
            'linear_z.weight': infer_dist("W3", self.hidden_layer_size, self.text_embeddings_size + self.img_embeddings_size), #no bias
            'linear_test.weight': infer_dist("W4", self.hidden_layer_size, self.hidden_layer_size), 'linear_test.bias': infer_dist("b4", self.hidden_layer_size),
            'linear.weight': infer_dist("W5", self.num_labels, self.hidden_layer_size), 'linear.bias': infer_dist("b5", self.num_labels)
            }

        lifted_module = pyro.random_module("net", net, dists)
        return lifted_module()

    '''

In [0]:
def model(image_embeddings, text_embeddings, labels):

    linear_h_text_w_prior = Laplace(loc=torch.zeros_like(net.linear_h_text.weight), scale=torch.ones_like(net.linear_h_text.weight))
    linear_h_image_w_prior = Laplace(loc=torch.zeros_like(net.linear_h_image.weight), scale=torch.ones_like(net.linear_h_image.weight))
    linear_z_w_prior = Laplace(loc=torch.zeros_like(net.linear_z.weight), scale=torch.ones_like(net.linear_z.weight))

    linear_test_w_prior = Laplace(loc=torch.zeros_like(net.linear_test.weight), scale=torch.ones_like(net.linear_test.weight))
    linear_test_b_prior = Laplace(loc=torch.zeros_like(net.linear_test.bias), scale=torch.ones_like(net.linear_test.bias))

    linear_w_prior = Laplace(loc=torch.zeros_like(net.linear.weight), scale=torch.ones_like(net.linear.weight))
    linear_b_prior = Laplace(loc=torch.zeros_like(net.linear.bias), scale=torch.ones_like(net.linear.bias))

    priors = {
        'linear_h_text.weight': linear_h_text_w_prior, #no bias
        'linear_h_image.weight': linear_h_image_w_prior, #no bias
        'linear_z.weight': linear_z_w_prior, #no bias
        'linear_test.weight': linear_test_w_prior, 
        'linear_test.bias': linear_test_b_prior,
        'linear.weight': linear_w_prior, 
        'linear.bias': linear_b_prior
    }  

    lifted_module = pyro.random_module("net", net, priors)
    lifted_reg_model = lifted_module().cuda()
    lhat = lifted_reg_model(image_embeddings, text_embeddings)
    pyro.sample("obs", Categorical(logits=lhat), obs=labels)

In [0]:
 softplus = torch.nn.Softplus()
 
 def guide(image_embeddings, text_embeddings, labels):

    # linear_h_text weight
    linear_h_text_w_mu = torch.randn_like(net.linear_h_text.weight)
    linear_h_text_w_sigma = torch.randn_like(net.linear_h_text.weight)
    linear_h_text_w_mu_param = pyro.param("linear_h_text_w_mu", linear_h_text_w_mu)
    linear_h_text_w_sigma_param = softplus(pyro.param("linear_h_text_w_sigma", linear_h_text_w_sigma))
    linear_h_text_w_dist = Laplace(loc = linear_h_text_w_mu_param, scale = linear_h_text_w_sigma_param)


    # linear_h_image weight
    linear_h_image_w_mu = torch.randn_like(net.linear_h_image.weight)
    linear_h_image_w_sigma = torch.randn_like(net.linear_h_image.weight)
    linear_h_image_w_mu_param = pyro.param("linear_h_image_w_mu", linear_h_image_w_mu)
    linear_h_image_w_sigma_param = softplus(pyro.param("linear_h_image_w_sigma", linear_h_image_w_sigma))
    linear_h_image_w_dist = Laplace(loc = linear_h_image_w_mu_param, scale = linear_h_image_w_sigma_param)


    # linear_z weight
    linear_z_w_mu = torch.randn_like(net.linear_z.weight)
    linear_z_w_sigma = torch.randn_like(net.linear_z.weight)
    linear_z_w_mu_param = pyro.param("linear_z_w_mu", linear_z_w_mu)
    linear_z_w_sigma_param = softplus(pyro.param("linear_z_w_sigma", linear_z_w_sigma))
    linear_z_w_dist = Laplace(loc = linear_z_w_mu_param, scale = linear_z_w_sigma_param)


    # linear_test weight
    linear_test_w_mu = torch.randn_like(net.linear_test.weight)
    linear_test_w_sigma = torch.randn_like(net.linear_test.weight)
    linear_test_w_mu_param = pyro.param("linear_test_w_mu", linear_test_w_mu)
    linear_test_w_sigma_param = softplus(pyro.param("linear_test_w_sigma", linear_test_w_sigma))
    linear_test_w_dist = Laplace(loc = linear_test_w_mu_param, scale = linear_test_w_sigma_param)

    
    # linear_test bias
    linear_test_b_mu = torch.randn_like(net.linear_test.bias)
    linear_test_b_sigma = torch.randn_like(net.linear_test.bias)
    linear_test_b_mu_param = pyro.param("linear_test_b_mu", linear_test_b_mu)
    linear_test_b_sigma_param = softplus(pyro.param("linear_test_b_sigma", linear_test_b_sigma))
    linear_test_b_dist = Laplace(loc = linear_test_b_mu_param, scale = linear_test_b_sigma_param)


    # linear weight
    linear_w_mu = torch.randn_like(net.linear.weight)
    linear_w_sigma = torch.randn_like(net.linear.weight)
    linear_w_mu_param = pyro.param("linear_w_mu", linear_w_mu)
    linear_w_sigma_param = softplus(pyro.param("linear_w_sigma", linear_w_sigma))
    linear_w_dist = Laplace(loc = linear_w_mu_param, scale = linear_w_sigma_param)

    
    # linear bias
    linear_b_mu = torch.randn_like(net.linear.bias)
    linear_b_sigma = torch.randn_like(net.linear.bias)
    linear_b_mu_param = pyro.param("linear_b_mu", linear_b_mu)
    linear_b_sigma_param = softplus(pyro.param("linear_b_sigma", linear_b_sigma))
    linear_b_dist = Laplace(loc = linear_b_mu_param, scale = linear_b_sigma_param)


    dists = {
        'linear_h_text.weight': linear_h_text_w_dist, #no bias
        'linear_h_image.weight': linear_h_image_w_dist, #no bias
        'linear_z.weight': linear_z_w_dist, #no bias
        'linear_test.weight': linear_test_w_dist, 
        'linear_test.bias': linear_test_b_dist,
        'linear.weight': linear_w_dist, 
        'linear.bias': linear_b_dist
    } 
    
    lifted_module = pyro.random_module("net", net, dists)
    return lifted_module()

In [0]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, trange
from sklearn import metrics
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import time
import datetime


def SetTrainDataloader_MM(Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor) :

  train_dataset = TensorDataset(Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor)
  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size =  batch_size)
  return(train_dataloader)


#source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
def Plot_Training_Epoch_Loss(epoch_loss_set) :

  sns.set(style='darkgrid')
  sns.set(font_scale=1.5)
  plt.rcParams["figure.figsize"] = (12,6)
  plt.plot( epoch_loss_set, 'b-o')
  plt.title("Training loss")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.savefig('Training_Epoch_Loss.png',bbox_inches='tight')
  plt.show()


#source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
def format_time(elapsed):
  '''
  Takes a time in seconds and returns a string hh:mm:ss
  '''
  # Round to the nearest second.
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))


def Train(net, train_dataloader, device, inference, learning_rate, epoch_loss_set) :
  
  net.cuda()
  for _ in trange( epochs, desc="Epoch"):
    
    net.train()
    epoch_loss = 0

    # Measure how long the training epoch takes.
    t0 = time.time()

    for step_num, batch_data in enumerate(train_dataloader):

      # Progress update every 30 batches.
      if step_num % 30 == 0 and not step_num == 0:
        elapsed =  format_time(time.time() - t0)
        print('  Batch : ',step_num, ' , Time elapsed : ',elapsed)

      samples_image, samples_text, labels = tuple(t.to(device) for t in batch_data)
      '''
      # define optimizer and loss function
      optimizer = Adam({"lr": learning_rate})
      loss_fn = pyro.infer.Trace_ELBO()

      # compute loss
      batch_loss = loss_fn(net.model, net.guide, samples_image.float(), samples_text.float(), labels)
      batch_loss.backward()
      
      # take a step and zero the parameter gradients
      optimizer.step()
      optimizer.zero_grad()
      '''
      batch_loss = inference.step(samples_image.float(), samples_text.float(), labels)
      epoch_loss += batch_loss

    avg_epoch_loss = epoch_loss/len( train_dataloader)
    print("\nTrain loss for epoch: ",avg_epoch_loss)
    print("\nTraining epoch took: {:}".format( format_time(time.time() - t0)))
    epoch_loss_set.append(avg_epoch_loss)

  #torch.save( model.state_dict(), "/content/drive/My Drive/dataset/model.pt")
  Plot_Training_Epoch_Loss(epoch_loss_set)

In [0]:
#Parameters

net = GMU_PP(num_maxout_units = 2, hidden_layer_size = 512, hidden_activation = None, dropout = 0.5).cuda()
label_names = Label_names
num_labels = 23
batch_size = 512
learning_rate = 0.01
epochs = 125
sigmoid_thresh = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = pd.DataFrame(0, index=['Recall','Precision','F_Score'], columns=['micro', 'macro', 'weighted', 'samples']).astype(float)
epoch_loss_set = []
train_dataloader =  SetTrainDataloader_MM(Data_train_tensor_image, Data_train_tensor_text, Labels_train_tensor[:,0])

# clear param store
pyro.clear_param_store()

# pyro svi, initialise losses
inference = SVI(model, guide, Adam({"lr": learning_rate}), loss=Trace_ELBO())
#inference = SVI(net.model, net.guide, Adam({"lr": learning_rate}), loss=Trace_ELBO())

Train(net, train_dataloader, device, inference, learning_rate, epoch_loss_set)

In [0]:
net = GMU_PP(num_maxout_units = 2, hidden_layer_size = 512, hidden_activation = None, dropout = 0.5).cuda()
label_names = Label_names
num_labels = 23
batch_size = 512
learning_rate = 0.01
epochs = 125
sigmoid_thresh = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = pd.DataFrame(0, index=['Recall','Precision','F_Score'], columns=['micro', 'macro', 'weighted', 'samples']).astype(float)
epoch_loss_set = []
train_dataloader =  SetTrainDataloader_MM(Data_train_tensor_image, Data_train_tensor_text, Labels_train_tensor)

# clear param store
pyro.clear_param_store()

# pyro svi, initialise losses
inference = SVI(model, guide, Adam({"lr": learning_rate}), loss=Trace_ELBO())
#inference = SVI(net.model, net.guide, Adam({"lr": learning_rate}), loss=Trace_ELBO())

Train(net, train_dataloader, device, inference, learning_rate, epoch_loss_set)

Epoch:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pyro/primitives.py:371: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)



RuntimeError: ignored

In [0]:
num_samples = 10

def predict(vgg_input, w2v_input):
  
    sampled_models = [guide(None, None, None) for _ in range(num_samples)]
    yhats = [model(vgg_input, w2v_input).data for model in sampled_models]
    mean = torch.mean(torch.stack(yhats), 0).cpu()
    predictions = np.argmax(mean.numpy(), axis=1)

    yhats_uncertainity = [F.log_softmax(model(vgg_input, w2v_input).data, 1).cpu().numpy() for model in sampled_models]
    uncertainity = np.asarray(yhats_uncertainity)

    return (predictions, uncertainity)



In [0]:
vgg_input = torch.tensor(dataset['vgg16_embeddings'][0:2]).cuda()
w2v_input = torch.tensor(dataset['w2v_embeddings'][0:2]).cuda()